## Mod 7 Lecture 1:  Discriminative vs. Generative Models

**Do this assignment in Google Colab**

Upload the FHV_072023.csv dataset into the 'sample_data' folder in Google Colab.  Recall this data is from July 1, 2025 to July 15, 2025.

Reorient yourself to the data [HERE](https://data.cityofnewyork.us/Transportation/2023-High-Volume-FHV-Trip-Data/u253-aew4/about_data)

**We will only work with the first 5000 rows**


In [1]:
# Imports
import pandas as pd 
import numpy as np

In [3]:
#Only look at the first 5,000 rows for speed
df = pd.read_csv("../Mod6/data/FHV_072023copy.csv", nrows=5000)
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0005,B03406,NaN,07/01/2023 05:34:30 PM,NaN,07/01/2023 05:37:48 PM,07/01/2023 05:44:45 PM,158,68,1.266,...,1.35,2.75,0.0,2.00,5.57,N,N,N,N,False
1,HV0003,B03404,B03404,07/01/2023 05:34:30 PM,07/01/2023 05:36:53 PM,07/01/2023 05:37:15 PM,07/01/2023 05:55:15 PM,162,234,2.350,...,1.52,2.75,0.0,3.28,13.38,N,N,NaN,N,False
2,HV0003,B03404,B03404,07/01/2023 05:34:30 PM,07/01/2023 05:35:17 PM,07/01/2023 05:35:52 PM,07/01/2023 05:44:27 PM,161,163,0.810,...,0.49,2.75,0.0,0.00,5.95,N,N,NaN,N,False
3,HV0003,B03404,B03404,07/01/2023 05:34:30 PM,07/01/2023 05:37:39 PM,07/01/2023 05:39:35 PM,07/01/2023 06:23:02 PM,122,229,15.470,...,5.17,2.75,0.0,0.00,54.46,N,N,NaN,N,True
4,HV0003,B03404,B03404,07/01/2023 05:34:30 PM,07/01/2023 05:36:06 PM,07/01/2023 05:36:39 PM,07/01/2023 05:45:06 PM,67,14,1.520,...,0.85,0.00,0.0,3.00,7.01,N,N,NaN,N,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   hvfhs_license_num     5000 non-null   object 
 1   dispatching_base_num  5000 non-null   object 
 2   originating_base_num  3675 non-null   object 
 3   request_datetime      5000 non-null   object 
 4   on_scene_datetime     3675 non-null   object 
 5   pickup_datetime       5000 non-null   object 
 6   dropoff_datetime      5000 non-null   object 
 7   PULocationID          5000 non-null   int64  
 8   DOLocationID          5000 non-null   int64  
 9   trip_miles            5000 non-null   float64
 10  trip_time             5000 non-null   object 
 11  base_passenger_fare   5000 non-null   float64
 12  tolls                 5000 non-null   float64
 13  bcf                   5000 non-null   float64
 14  sales_tax             5000 non-null   float64
 15  congestion_surcharge 

## Prompt 1 (Using AI for Entire Workflow) (We Do)

Copy and paste Prompt 1 into Gemini (within Google Colab) and paste the output below.  Just accept the output as is and run it step-by-step (or autorun it)

**It may take awhile to run!** 
*While it is running discuss*: 
- What do you expect the AI output to be (how many lines of code, all coding cells or mix of code and markdown cells, etc.)? 
- Is there any part of this prompt you would change (delete or add)?
- What parts of this prompt would you want to evaluate the output of first?
- Do you think everyone will get the same output?

“**Context**: I’m using the first 5,000 rows of a single NYC HVFHV CSVs with data dated for July 1, 2023 and July 15, 2023. I want an MLR predicting total_amount from trip_miles, trip_time, and engineered features. 

**Artifacts**: Produce a single Python code cell that: (1) reads one local CSVs I’ll upload; (2) selects columns: pickup_datetime, trip_miles, trip_time, base_passenger_fare, tips, total_amount; (3) concatenates; (4) drops rows with missing in these columns; (5) builds a Pipeline with imputer→scaler→LinearRegression; (6) fits train/test (80/20, random_state=42); (7) prints MAE/RMSE. 

**Rules**: No network calls, use pandas/sklearn only, don’t rename columns. 

**Evaluation**: Add 3 asserts (no NA, >1,000 rows, numeric dtypes) and print shapes.”

## Prompt 2 (Using AI for Feature Engineering) - You Do

Copy and paste Prompt 2 into Gemini (within Google Colab) and paste the output below this cell

“Suggest 6 feature ideas for predicting total_amount using HVFHV trips with columns: trip_miles, trip_time, base_passenger_fare, tips, total_amount. Label each as: linear, interaction, polynomial, or bucket. Explain expected direction. Output as a markdown table; no code.”

## Prompt 3 (Using AI for Diagnostics) - You Do 

Copy and paste Prompt 2 into Gemini (within Google Colab) and paste the output below this cell. Just accept the output as is and run it step-by-step (or autorun it)

“Generate a Python cell (no external calls) to: (1) plot residuals vs predicted for my fitted pipe and X_test, y_test; (2) print top 10 coefficients with feature names from pipe.named_steps["pre"].get_feature_names_out() aligned to pipe.named_steps["reg"].coef_; (3) sort by absolute value; (4) warn if any absolute residual > 3×RMSE.”

## Reflection (Share Out)

1) What prompt(s) gave you the "best" output 
2) What were some issues you saw in the prompt output? 
3) **How did the MLR model GenAI gave you compare to the model you did in Mod6L10/Mod6L11?** VERY IMPORTANT
4) If you all didn't get the same responses, what do you think is a limitation of GenAI? (think "memory")
5) Are there any ethical issues with the prompt output?